We will implement a simple flip-coin game using the Hadamard gate in three different ways. [The Hadamard gate](/qutips/the-hadamard-gate.html) is the simplest method for generating a random bit in a qubit. If you then attempt to measure that qubit the quantum state collapses into either a 0 or a 1 with equal probability, generating a single random bit.

In all three scripts we will use a small input method which will aks for your guess, and store your input into variable **guess**. Also, we will set a constant **tosses** to value 15 (number of tries).

#### Approach-1

In this approach we will use Aer simulator to measure state vector. We will create a circuit with only one qubit, and we will execute it and measure the state vector for the number of tosses. Number of heads and tails will be increased by the result state vector. 

```python
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

qc = QuantumCircuit(1)  # quantum circuit with one qubit
qc.h(0)  # applying Hadamard gate
qc.measure_all()

for i in range(tosses):
    backend = Aer.get_backend('statevector_simulator')
    result = execute(qc, backend).result()

    coin = result.get_statevector()

    if np.all(coin == np.array([0., 1.])):
        heads = heads + 1
    else:
        tails = tails + 1

```

#### Approach-2

In this second approach we will create again a single qubit circuit, but this time we will run it on IBMQ. We will execute it on **ibmq_essex** device, and set number of shots to out constant tosses.

At the end we will measure a number of zero's and one's in the result as the final number of heads and tails.

```python
from qiskit import QuantumCircuit
from qiskit import IBMQ
from qiskit.compiler import transpile, assemble

IBMQ.load_account()

qc = QuantumCircuit(1)  # quantum circuit with one qubit
qc.h(0)  # applying Hadamard gate
qc.measure_all()

# Execute on IBMQ
provider = IBMQ.get_provider(group='open')
backend = provider.get_backend('ibmq_essex')
transpiled_circs = transpile(qc, backend=backend)
qobjs = assemble(transpiled_circs, backend=backend, shots=tosses)
job_info = backend.run(qobjs)

# Measure the results
result = job_info.result().get_counts()
heads = result['0']
tails = result['1']
```    

#### Approach-3

In the last third approach we will create a list with **tosses** number of elements. For each element in a list we will create a one qubit circuit, apply a Hadamard gate on it, and measure the result. 

This list of circuits will be executed on **ibmq_london** device, with default number of shots (1024).

For each circuit in the list we will count the number of |0> and |1> and based on that increase the number of heads and tails in the final result.

```python
from qiskit import QuantumCircuit
from qiskit import IBMQ
from qiskit.compiler import transpile, assemble

qc_list = []

IBMQ.load_account()

for i in range(tosses):
    qc = QuantumCircuit(1)  # quantum circuit with one qubit
    qc.h(0)  # applying Hadamard gate
    qc.measure_all()
    qc_list.append(qc)

# Execute a list of circuits
provider = IBMQ.get_provider(group='open')
backend = provider.get_backend('ibmq_london')
transpiled_circs = transpile(qc_list, backend=backend)
qobjs = assemble(transpiled_circs, backend=backend)
job_info = backend.run(qobjs)

# Measure the results
for circ_index in range(len(transpiled_circs)):
    result = job_info.result().get_counts(transpiled_circs[circ_index])
    print(result)
    heads_count = result['0']
    tails_count = result['1']
    if heads_count > tails_count:
        heads = heads + 1
    else:
        tails = tails + 1
```    

All fully functional code is available on those links: [Approach-1](/code/flip_coin_game.py), [Approach-2](/code/flip_coin_game_2.py) and [Approach-3](/code/flip_coin_game_3.py).